## Python to clips generator

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('llibres.csv')
df1.drop(columns='Unnamed: 0', inplace =True)
df1.drop(columns='Unnamed: 0.1', inplace =True)
df1.drop(columns='Unnamed: 0.2', inplace =True)
df1.drop(columns='Unnamed: 0.3', inplace =True)

df2 = pd.read_csv('authors_on_cluster.csv')
df2.drop(columns='Unnamed: 0', inplace =True)
df2.drop(columns='Unnamed: 0.1', inplace =True)

In [5]:
# Create a dictionary of subgenres and subsubgenres
literary_genres = {
    "Realistic": ["social", "psychological", "manners"],
    "Historical": ["medieval", "ancient civilization", "world war"],
    "Romantic": ["contemporary romance", "historical romance", "paranormal romance"],
    "Science Fiction": ["space opera", "dystopian fiction", "cyberpunk"],
    "Fantasy": ["high fantasy", "low fantasy", "urban fantasy"],
    "Adventure": ["pirate", "travel", "exploration"]
}

#estilos = ['Narrativa en primera persona', 'Narrativa en tercera persona', 'Monólogo interior', 'Estilo experimental', 'Estilo epistolar', 'Estilo minimalista', 'Estilo descriptivo', 'Narrativa en segunda persona']


In [6]:
def conocimiento_a_clips(libros , autores, fichero = 'instances.clp', generos = {}):
    
    #Creación de diccionarios con valor identificativo para cada instancia.
    dicc_libros = {}
    for index, row in libros.iterrows():
        dicc_libros[row['title']] = f'Libro_Class_{index}'
    
    dicc_autores = {}
    for index, row in autores.iterrows():
        dicc_autores[row['authors']] = f'Autor_Class_{index}'
    
    dicc_generos = {}
    dicc_subgeneros = {}

    for index,value in enumerate(generos.keys()):
        dicc_generos[value] = f'Genero_Class_{index}'
    
        for index_2,value_2 in enumerate(generos[value]):
            dicc_subgeneros[value_2] = f'Subgenero_Class_{index*len(generos[value]) +index_2}'
        
    #Abrir un archivo en modo de escritura con codificación utf-8. Si el archivo no existe, será creado.
    with open(fichero, 'w', encoding='utf-8') as file:
        file.write('(definstances instances\n')
        
        #GENEROS
        for index, key in enumerate(generos.keys()):

            genero_instance = f"""
        ([{dicc_generos[key]}] of Genero
            (nombre "{key}")
        )"""

            # Escribir la instancia en el archivo
            file.write(genero_instance + '\n')
        
        #SUBGENEROS
        for key in generos.keys():
            for value in generos[key]:
                
                #Supergenero
                supergenero = dicc_generos[key]

                subgenero_instance = f"""
        ([{dicc_subgeneros[value]}] of Subgenero

            (nombre "{value}")
            (subgenero_de [{supergenero}])
        )"""
                # Escribir la instancia en el archivo
                file.write(subgenero_instance + '\n')
        
        #LIBROS
        for index, row in libros.iterrows():

            libro_instance = f"""
        ([{dicc_libros[row['title']]}] of Libro
            (genero_de [{dicc_generos[row['genre']]}])
            (subgenero_de [{dicc_subgeneros[row['subgenre']]}])
            (nombre "{row['title']}")
            (escrito_por [{dicc_autores[row['authors']]}])
            (publicacion {int(row['first_publish_year'])})
            (valoracion {row['Rating (0-5)']})
            (paginas {row['Pages']})
            (popularidad {row['Weeks on Best-Seller List']})
            (complejidad {row['Number of Characters']})
            (estilo "{row['Estilo de Escritura']}")
            (publico "{row['Publico']}")
        )"""
            # Escribir la instancia en el archivo
            file.write(libro_instance + '\n')

        #AUTORES
        for index, row in autores.iterrows():
            # Lista de libros
            filtered_df = libros[libros['authors'] == row['authors']]
            
            author_instance = f"""
        ([Autor_Class_{index}] of Autor
            (escribe_subgenero [{dicc_subgeneros[row['Most Common Subgenre']]}])
            (nombre "{row['authors']}")
            (cluster "{row['Cluster']}")
        )"""
            # Escribir la instancia en el archivo
            file.write(author_instance + '\n')
        
        # Escribir el final del bloque definstances
        file.write(')\n')


In [8]:
conocimiento_a_clips(libros = df1,autores = df2, generos = literary_genres)